<a href="https://colab.research.google.com/github/hannape/Gruba-kreska/blob/main/Funkcja_wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
def funkcja_wczytanie_danych(path_test1618_txt, path_train161718_txt, path_test1618_wav, path_train161718_wav, \
                             balance_types, balance_ratios, chunk_length_ms, chunk_overlap, calls_0, calls_1, \
                             calls_unknown, tolerance, valid_set, test_rec_to_cut, columns_dataframe):

  '''
  Jest to funkcja zawierająca cały potrzebny kod z pliku Wczytanie_danych, bez zbędnych printów itd.
  Args: 
  te wszystkie parametry wejściowe co powyżej, nie chce mi się ich rozpisywać...
  Returns:
  file_names_train_set, ind_for_train_set, result_dataframe_train - nazwy nagrań, indeksy chunksów z danego nagrania wybrane do zbioru treningowego (96) + dataframe treningowy
  file_names_valid_set, ind_for_valid_set, result_dataframe_valid - nazwy nagrań, indeksy chunksów z danego nagrania wybrane do zbioru walidacyjnego (8) + dataframe walidacyjny
  file_names_test_old, ind_for_test_old, result_dataframe_test_old - nazwy nagrań, indeksy chunksów z danego nagrania wybrane do zbioru testowego (20) + dataframe test old
  file_names_test_new, ind_for_test_new, result_dataframe_test_new - nazwy nagrań, indeksy chunksów z danego nagrania wybrane do zbioru testowego (18) + dataframe test new

  Struktura dataframe:
    columns_dataframe = ['chunk_ids', 'chunk_start', 'chunk_end', 'has_bird', 'chunks_species', 'call_id', 'has_unknown', 'has_noise']
    result_dataframe_X = pd.DataFrame(data = X, index=file_names_X, columns = columns_dataframe)
  '''
  ## IMPORTY

  import os
  import contextlib
  import numpy as np
  import wave
  import math
  import pandas as pd
  import random
  from timeit import default_timer as timer

  ### FUNKCJE 

  def my_read_labels(label_name,path_txt):  # ta sama funkcja co w 1. poprawianie danych
      '''Wczytuje etykiety czasowe z pliku labels.txt w folderze train.
      
      Returns:
          ndarray: Tablica z etykietami czasowymi zawierająca kolumny:  sekunda początku dźwięku, sekunda końca dźwięku, gatunek.
      '''
      labels = []
      with open(os.path.join(path_txt, label_name + '.txt'), 'r') as file:
          text = file.read()
          for line in text.split('\n')[0:]:
              if len(line) > 1:
                  start, stop, spec = line.split('\t')
                  #print(start)
                  start, stop, spec = float(start), float(stop), str(spec),
                  labels.append([start, stop, spec])
      return np.array(labels)

  def my_check_labels(second, chunk_length_s, labels, tol=0.):
      '''Sprawdza czy w ramce czasowej [second, second + chunk_length_s] znajduje się coś wg etykiet `labels`.
      
      Args:
          second (float): Sekunda nagrania.
          labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a 2ga - koniec nagrania.
          tol (float): Tolerancja na brzegach fragmentu. Dźwięk, żeby był uznany, musi się kończyć po czasie `second+tol`
              lub zaczynać przed czasem `second+chunk_length-tol`.
      Returns:
          bool: Czy w ramce czasowej jest etykieta - co najmniej 4 ms labelu.
  ''' 

    # jeśli początek etykiety zawiera się w chunku, najpóźniej 4ms przed końcem chunka
    # jeśli koniec etykiety zawiera się w chunku, i najwcześniej 4 ms po rozpoczęciu chunka
    # jeśli początek etykiety zaczyna się rpzed, a koniec po (w przypadku głosów dłuższych niż chunk length, b. rzadkie przypadki dla 0.5 sek)
      return (float(labels[0]) >= second and float(labels[0]) < second + chunk_length_s - tol) or \
            (float(labels[1]) < second + chunk_length_s and float(labels[1]) > second + tol) or \
            (float(labels[0]) < second and float(labels[1]) > second + chunk_length_s)              
            #and (labels[2] in calls_to_cut)    # nie sprawdzajmy tu jaka to etykieta jest

  def my_map_seconds_to_y(labels, recording_duration, calls_of_interest, calls_to_cut, calls_unknown):  ## Chcemy 500 ms z 150 ms overlapem
      '''Tworzy etykiety dla każdego kolejnego chunksa. 1 gdy co najmniej 4 ms etykiety się znajdują w chunksie (gdy urwane dźwięki na brzegach, <4ms - to 0).
      
      Args:
          labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a druga - koniec nagrania.
      Returns:
          ndarray: Tablica z binarnymi etykietami.
      '''
      #STARE: calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']
      
      
      duration_in_ms = recording_duration*1000
      nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)
      #print("ilość chunksów w tym nagraniu: ", math.ceil(nr_of_chunks) )
      
      
      y = [0] * math.ceil(nr_of_chunks)             # recording_duration 
      y_restrictive = [0] * math.ceil(nr_of_chunks) # recording_duration
      chunks_start, chunks_end = [0] * math.ceil(nr_of_chunks), [0] * math.ceil(nr_of_chunks)      
      has_unknown, has_noise = [0] * math.ceil(nr_of_chunks), [0] * math.ceil(nr_of_chunks)
      call_id = [[] for _ in range(math.ceil(nr_of_chunks))]
      chunks_species = [[] for _ in range(math.ceil(nr_of_chunks))]
      
      #print(np.shape(labels))
      for s in range(math.ceil(nr_of_chunks)):
          chunks_start[s] = s * ((chunk_length_ms-chunk_overlap) / 1000)  # czyli s * 0.35
          chunks_end[s] = chunks_start[s] + chunk_length_ms/1000 
          for ind,l in enumerate(labels):
              #print('label_index: ',ind, ' a label to: ', l[2] )
              if my_check_labels(chunks_start[s], chunk_length_ms/1000, l):
                  #print(s*0.35)
                  if l[2] in calls_to_cut:
                    has_noise[s] = 1
                  if l[2] in calls_unknown:
                    has_unknown[s] = 1
                  if l[2] in calls_of_interest:  
                    y[s] = 1 
  
              if my_check_labels(chunks_start[s], chunk_length_ms/1000, l, 0.004): #z tolerancją 
                  
                  if l[2] in calls_to_cut:
                    chunks_species[s].append(l[2])
                  if l[2] in calls_unknown:
                    has_unknown[s] = 1
                    chunks_species[s].append(l[2])
                  if l[2] in calls_of_interest:  
                    y_restrictive[s] = 1  
                    chunks_species[s].append(l[2])  
                    call_id[s].append(ind)
                
                        
          if y[s] != y_restrictive[s] and l[2] in calls_of_interest:
              #print('próbka ', s, 'zaczynajaca sie ', chunks_start[s],', jest unknown, bo za mały fragment głosu nas interesującego')
              #print('y[s]:', y[s],'yres:', y_restrictive[s] )
              y[s] = 0    # jeśli mniej niż 4 ms fragment, to zakładamy że nie ma głosu, i dajemy unknown także, by nie karać za ewentualne znalezienie tego głosu
              has_unknown[s] = 1
      #print('Rozmiar ',np.shape([s, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown]))
      return s+1, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown, has_noise    

  def my_load_wav(path_wav, path_txt, recordings_incl = None, recordings_excl = None):   # dłuższa funkcja niż w 1.

    recording_labels_all = []
    in_which_recording = []
    file_names = []

    # wszystkie nagrania
    if recordings_incl == None and recordings_excl == None:                                        
      rec_files = sorted([file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')])

    # nagrania tylko te w podanym zbiorze (walid)
    if recordings_incl != None:                                                                     
      rec_files_all = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]
      recordings_incl = [s + '.wav' for s in recordings_incl]
      rec_files = sorted(list( set.intersection(set(rec_files_all), set(recordings_incl))))

    # nagrania oprócz nagrań w podanym zbiorze (train)  
    if recordings_excl != None:                                                                     
      rec_files_all = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]
      recordings_excl = [s + '.wav' for s in recordings_excl]
      rec_files = sorted(list(set(rec_files_all) - set(recordings_excl))  )
    
    X_matrix = [[] for _ in range(np.size(rec_files))]
    file_names = [[] for _ in range(np.size(rec_files))]

    for ind, file_name in enumerate(rec_files):
        
        print("------------Analiza nagrania: " + file_name + "-----------")
        recording_id = str(file_name.split('.')[0])
        
        fname = path_wav + file_name      #print(fname)
        with contextlib.closing(wave.open(fname,'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            duration = frames / float(rate)
            recording_duration = math.ceil(duration)
            #print("Czas trwania nagrania - w sekundach: " + str(recording_duration))
            #print("rate:", rate)
      
        recording_id = (file_name.split('.')[0])       #print(recording_id)
        recording_labels = my_read_labels(recording_id, path_txt)    

        #print("Duration:", duration)
        #y_binary = my_map_seconds_to_y(recording_labels, duration)
        chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown, has_noise  = my_map_seconds_to_y(recording_labels, duration, calls_1, calls_0, calls_unknown)
        chunk_ids = range(chunk_id)
        chunk_start = rate*np.array(chunk_start)
        chunk_start = [round(num) for num in chunk_start]
        chunk_end = rate*np.array(chunk_end)
        chunk_end = [round(num) for num in chunk_end]
        X_matrix[ind] = [chunk_ids, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown, has_noise]  # !!!! chunk start i end przerzucone na sample !!!!!
        file_names[ind] = file_name

    return X_matrix , file_names #chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown

  def find_ratio(recordings_names, path_txt):
    count_s,count_k, count_d, count_r, count_positives = 0,0,0,0,0
    for rec_name in recordings_names:
      recording_labels = my_read_labels(rec_name, path_txt)  
      #print(recording_labels)
      counted_labels = Counter(recording_labels[:,2])

      #print(counted_labels)
      for calls in labels_s:
        count_s += counted_labels[calls]

      for calls in labels_k:
        count_k += counted_labels[calls]

      for calls in labels_d:
        count_d += counted_labels[calls]   

      for calls in labels_r:
        count_r += counted_labels[calls] 

      for calls in calls_1:
        count_positives += counted_labels[calls] 

    return [count_s, count_k, count_d, count_r, count_positives]  

  def create_set_ind(result_dataframe, balance, ratios = None):
    
    #print(balance)
    file_names = list(result_dataframe.index)
    #print(len(file_names))
    ind_chosen = [[] for _ in range(len(file_names))]
    neg_random_numb = 0;
    all_pos = 0
    all_chunks = 0


    for file_i, file_name in enumerate(file_names):
      #print(file_i, ' nagranie ', file_name)
      
      indices_hasbird = [i for i, j in enumerate(result_dataframe['has_bird'][file_i]) if j == 1]
      indices_hasnoise = [i for i, j in enumerate(result_dataframe['has_noise'][file_i]) if j == 1]
      indices_unknown = [i for i, j in enumerate(result_dataframe['has_unknown'][file_i]) if j == 1]
      indices_all = result_dataframe['chunk_ids'][file_i]
      pos = indices_hasbird


      if balance == 'full_rec':
        
        ind_chosen[file_i] = list(set(indices_all) - (set(indices_unknown) - set(indices_hasbird)) )  
        # te które mają ptaka i unknown, to zostają. W każdym innym przypadku, unknown wylatuje z setu
        #print(np.shape(ind_chosen))
        all_pos += len(pos)
        all_chunks += len(ind_chosen[file_i])
        #print('pos: ', len(indices_hasbird))  # pozytywne
        neg_labels = list(set(indices_hasnoise) - set(indices_hasbird +indices_unknown))
        #print('neg hałas:' , len(neg_labels))
        #print('ind_chosen:' , len(ind_chosen[file_i]))

      if balance == 'balanced' or balance == 'valid':
        
        #print('pos: ', len(indices_hasbird))  # pozytywne
        
        neg_labels = list(set(indices_hasnoise) - set(indices_hasbird +indices_unknown))
        #print('neg hałas:' , len(neg_labels))
        #print(len(list(set(indices_hasnoise) - set(indices_hasbird +indices_unknown)))) # z samym hałasem, bez unknown czy ptaków. Czyli 0
        #temp =  list(set(indices_hasbird + indices_hasnoise))

        
        
        if (len(pos) - len(neg_labels)>=0):   
          neg_random_numb = len(pos) - len(neg_labels)
          if (neg_random_numb < ratios[1]):
            neg_random_numb = ratios[1]
          if neg_random_numb + len(neg_labels) < ratios[0]:
            neg_random_numb = ratios[0] - len(neg_labels)
        else:
          neg_random_numb = ratios[1]

        #print('neg random: ', neg_random_numb)

        neg_random_set = list(set(indices_all) - (set(indices_unknown + indices_hasbird + indices_hasnoise)))


        random.seed(667)  # zmienione, z file_i
        neg_random = random.sample(neg_random_set, neg_random_numb)
        #print(neg_random[0:5])
        ind_chosen[file_i] = sorted(list(set(indices_hasbird + neg_labels + neg_random)))

        all_pos += len(pos)
        all_chunks += len(ind_chosen[file_i])  

    print('wszystkich chunków: ', all_chunks,', w tym ', all_pos, ' pozytywnych. Czyli ', 100* round(all_pos/all_chunks,4),' % pozytywnych w zbiorze', balance )
    return file_names, ind_chosen

  # trening i walid  

  print('*************  Train  ***********')

  X_train, file_names_train = my_load_wav(path_train161718_wav, path_train161718_txt, None, valid_set) 
  result_dataframe_train = pd.DataFrame(data = X_train, index=file_names_train, columns = columns_dataframe)
  file_names_train_set, ind_for_train_set = create_set_ind(result_dataframe_train, balance_types[1], balance_ratios[1])

  print('*************  Valid  ***********')

  X_valid, file_names_valid = my_load_wav(path_train161718_wav, path_train161718_txt, valid_set, None)   
  result_dataframe_valid = pd.DataFrame(data = X_valid, index=file_names_valid, columns = columns_dataframe)  
  file_names_valid_set, ind_for_valid_set = create_set_ind(result_dataframe_valid, balance_types[2], balance_ratios[2])

  # testy

  print('*************  Test old  ***********')

  X_test_old, file_names_test_old = my_load_wav(path_test1618_wav, path_test1618_txt) 
  result_dataframe_test_old = pd.DataFrame(data = X_test_old, index=file_names_test_old, columns = columns_dataframe)
  file_names_test_set_old, ind_for_test_set_old = create_set_ind(result_dataframe_test_old, balance_types[0], balance_ratios[0])

  print('*************  Test new  ***********')
 
  X_test_new, file_names_test_new = my_load_wav(path_test1618_wav, path_test1618_txt, None, test_rec_to_cut)
  result_dataframe_test_new = pd.DataFrame(data = X_test_new, index=file_names_test_new, columns = columns_dataframe)
  file_names_test_set_new, ind_for_test_set_new = create_set_ind(result_dataframe_test_new, balance_types[0], balance_ratios[0])


  return [[file_names_train_set, ind_for_train_set, result_dataframe_train],[file_names_valid_set, ind_for_valid_set, result_dataframe_valid],
          [file_names_test_set_old, ind_for_test_set_old, result_dataframe_test_old], [file_names_test_set_new, ind_for_test_set_new, result_dataframe_test_new]]